In [9]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""],
)

loader = Docx2txtLoader('./data/tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

print(len(document_list))
print(f'각 청크 길이 {list(len(text.page_content) for text in document_list)}')

225
각 청크 길이 [1466, 1421, 1427, 1396, 1398, 1481, 1251, 1483, 1471, 1480, 1479, 1425, 1494, 1458, 1492, 1482, 1493, 1429, 1363, 1437, 1463, 1450, 1481, 1437, 1441, 1456, 1433, 1319, 1458, 1496, 1461, 1382, 1384, 1479, 1494, 1435, 1376, 1479, 1470, 1481, 1333, 1412, 1477, 1014, 1456, 1362, 1446, 1386, 1055, 1467, 1361, 1493, 1473, 1493, 1428, 1443, 1470, 1486, 1290, 1442, 1421, 1496, 1478, 1288, 1492, 1458, 1441, 1237, 1476, 1416, 1431, 1429, 1445, 1460, 1483, 1399, 1484, 1481, 1333, 1308, 1385, 1479, 1495, 1386, 1375, 1345, 1353, 1382, 1446, 1356, 1409, 1441, 1443, 1431, 1494, 1226, 1413, 1468, 1396, 1455, 1488, 1496, 1446, 1353, 1397, 1427, 1443, 1440, 1478, 1443, 1436, 1458, 1360, 1434, 1429, 1443, 1412, 1494, 1414, 1478, 1471, 1373, 1492, 1429, 1493, 1377, 1438, 1389, 1305, 1446, 1160, 1403, 1465, 1371, 1342, 1427, 1235, 1363, 1460, 1387, 1277, 1473, 1386, 1410, 1460, 1457, 1489, 1488, 1335, 1454, 1411, 1488, 1482, 1448, 1265, 1412, 1352, 1377, 1459, 1451, 1288, 1450, 1481, 1459, 147

In [12]:
# 임베딩
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [13]:
# from langchain_chroma import Chroma

# # 데이터 처음 저장할 때
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     collection_name='chroma-tax',
#     persist_directory='./chroma'
# )

In [14]:
%pip install -qU langchain-pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
%pip install -qU pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pincone_api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=pincone_api_key)
pinecone_index = pc.Index(index_name)

vectorstore = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [17]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가?"
query = "연봉 5천만원 직장인 소득세는 얼마인가?"

In [18]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
retriever.invoke(query)

[]

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [20]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [21]:
# from langchain.prompts import PromptTemplate

# prompt = PromptTemplate(
#     input_variables=['context', 'question'],
#     template=""" 
# [Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해 주세요.  

# [Context]
# {context}

# Question: {question}
# """
# )

In [22]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)

In [23]:
ai_message = qa_chain.invoke({"query": query})

In [24]:
ai_message

{'query': '연봉 5천만원 직장인 소득세는 얼마인가?',
 'result': '연봉 5천만원 직장인의 소득세는 구체적인 정보와 변수를 고려해야 정확히 계산할 수 있습니다. 일반적인 한국 소득세율에 따라 계산할 경우, 세액공제와 감면 조건에 따라 다르겠지만 대략적인 세율을 적용할 수 있습니다. 정확한 금액은 국세청의 소득세 계산기를 참조하시기 바랍니다.'}

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt2 = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전 : {dictionary}

    질문: {{question}}

""")

dictionary_chain = prompt2 | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [26]:
# new_question = dictionary_chain.invoke({"question": query})
# new_question

In [27]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가?"
query = "연봉 5천만원 직장인의 소득세는 얼마인가?"

In [28]:
ai_message = tax_chain.invoke({"question": query})

In [29]:
ai_message

{'query': '사용자의 질문을 변경할 필요가 없습니다. 현재의 질문은 사전에서 제공된 표현과 관련이 없기 때문에 그대로 사용할 수 있습니다.',
 'result': "I don't know. The provided context does not contain information relevant to answering your question."}